In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import napari
from FoodSeg103_data_handler import *
from scribbles_creator import create_even_scribbles
from convpaint_helpers import selfpred_convpaint

c:\Users\roman\anaconda3\envs\ilastik2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Create scribbles

Load the ground truths as batch

In [19]:
img_nums = [n for n in range(0, 4900, 500)] #2750 #1234 #2314
gts = load_food_batch(img_nums, load_images=False)[1]
num_imgs = len(gts)
print(f"Total images: {num_imgs}")

Total images: 10


Check and filter for resolution

In [20]:
resol = {im_num: np.prod(gt.shape) for im_num, gt in gts.items()}

# Check for images with resolution over a certain threshold
thresh = 200_000
print("Resolutions:")
for im_num, r in resol.items(): print(f"{im_num:4d}: {r:9,d} pixels{' (!)' if r > thresh else ''}")

Resolutions:
   0:   196,608 pixels
 500:   307,200 pixels (!)
1000:   196,608 pixels
1500:   133,660 pixels
2000:   172,544 pixels
2500:   196,608 pixels
3000:   174,080 pixels
3500:   196,608 pixels
4000:   196,608 pixels
4500: 3,145,728 pixels (!)


In [21]:
# Filter out those images
gts = {im_num: gt for im_num, gt in gts.items() if resol[im_num] < thresh}
num_imgs = len(gts)
print(f"Images with resolution under {thresh:,d}: {num_imgs}")
resol_new = {im_num: np.prod(gt.shape) for im_num, gt in gts.items()}
for im_num, r in resol_new.items(): print(f"{im_num:4d}: {r:7,d} pixels")

Images with resolution under 200,000: 8
   0: 196,608 pixels
1000: 196,608 pixels
1500: 133,660 pixels
2000: 172,544 pixels
2500: 196,608 pixels
3000: 174,080 pixels
3500: 196,608 pixels
4000: 196,608 pixels


Define scribbles parameters

In [22]:
# Which scribbles to create
bins = [0.05, 0.3, 1.5]
suff = ["w3"] #standing for width 3
sq_scaling = 100
scribble_width = 3
scribbles_seed = 1

# Where to save the scribbles
folder_path = "./FoodSeg103_results/data/run02"

save_res = True
show_res = False
print_steps = True

Loop and create scribbles

In [23]:
percentages = {}
for bin in bins:
    percentages[bin] = []
    for img_num, gt in gts.items():
        for s in suff:
            np.random.seed(scribbles_seed)
            print(f"IMG {img_num}: bin {bin}, suff {s}")
            scribbles, perc_labelled = create_food_scribble(gt, folder_path, img_num, bin=bin, sq_scaling=sq_scaling, mode="all",
                                                            save_res=save_res, suff=s, show_res=show_res, image=None, print_steps=print_steps, scribble_width=scribble_width)
            percentages[bin].append(perc_labelled)
            print(f"Annotation percentage: {perc_labelled:2.2f}%\n")

IMG 0: 0.05_w3
CLASS 1:
sk_max_pix: 7.88, sq_size: 44, sk_pix_range: (7, 88)
Adjusting square size and range to 22 (7, 88)
Adjusting square size and range to 11 (5, 88)
   prim_sk_squares pix: 6 = 0.01%
Adjusting square size and range to 22 (7, 88)
Adjusting square size and range to 11 (5, 88)
   sec_sk_squares pix: 6 = 0.01%
lines_max_pix: 7.88, line_pix_range: (7, 88)
   lines pix: 7 = 0.01%
TOTAL pix: 19 = 0.04%
CLASS 67:
sk_max_pix: 12.20, sq_size: 44, sk_pix_range: (12, 88)
Adjusting square size and range to 22 (11, 88)
   prim_sk_squares pix: 12 = 0.02%
Adjusting square size and range to 22 (11, 88)
Adjusting square size and range to 11 (5, 88)
   sec_sk_squares pix: 11 = 0.02%
lines_max_pix: 12.20, line_pix_range: (12, 88)
Adjusting line range to (6, 176)
   lines pix: 8 = 0.01%
TOTAL pix: 31 = 0.04%
CLASS 91:
sk_max_pix: 4.85, sq_size: 44, sk_pix_range: (4, 88)
Adjusting square size and range to 22 (4, 88)
   prim_sk_squares pix: 4 = 0.01%
Adjusting square size and range to 22 

Quick report of labelling percentages:

In [24]:
for bin, perc_list in percentages.items():
    print(f"labelled percentage in bin {bin}: mean = {np.mean(perc_list):.4f}, std = {np.std(perc_list):.4f}, range = {np.min(perc_list):.4f} - {np.max(perc_list):.4f}")

labelled percentage in bin 0.05: mean = 0.1873, std = 0.0161, range = 0.1572 - 0.2160
labelled percentage in bin 0.3: mean = 1.0110, std = 0.0400, range = 0.9232 - 1.0530
labelled percentage in bin 1.5: mean = 5.3396, std = 0.1640, range = 4.9805 - 5.4998


Show the last created scribble with its ground truth, and all ground truths (optionally)

In [25]:
v = napari.Viewer()
v.add_labels(gt, name="Ground Truth")
v.add_labels(scribbles, name="Scribbles")

<Labels layer 'Scribbles' at 0x22834ef0490>

In [58]:
for g in gts:
    v.add_labels(gts[g], name=f"GT {g}")